<a href="https://colab.research.google.com/github/mike1336git/colab_notebook/blob/main/with_js/js012_mixLJMD3D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### simulator( html + css + js ) + control( python )


In [1]:
#@title js012_mixLJMD3D / def exec_html_js() ... exec me first

# def exec_html_js()

import IPython
from IPython.display import display, HTML
from google.colab.output import eval_js

def exec_html_js():
  htm = HTML('''


<!DOCTYPE html>
<html>
<head>
<meta charset="utf-8">
<title>js012_mixLJMD3D</title>
<script type="text/javascript">

// %%%%%%%%%%%%%%%%%%%%  javaScript  %%%%%%%%%%%%%%%%%%%%

'use strict';

/* --------------------
//
//  js012_mixLJMD3D
//    Copyright(C) 2017-2023 Mitsuru Ikeuchi
//    Released under the MIT license ( https://opensource.org/licenses/MIT )
//
//    ver 0.0.0  2017.05.06 created, last updated on 2018.11.13
//    ver 0.0.1  2019.01.12 v1, last updated on 2021.05.01
//    ver 0.0.2  2021.10.29 v2, last updated on 2021.10.29
//    ver 0.0.3  2023.02.23 v3, last updated on 2023.08.16
//
// --------------------  molecular dynamics 2D
//
//  molecular dynamics 2D
//    method: velocity Verlet Algorithm
//      (1) vi = vi + (Fi/mi)*(0.5dt)
//      (2) ri = ri + vi*dt
//      (3) calculation Fi <- {r1,r2,...,rn} Fi=sum(Fij,j=1 to n),Fij=F(ri-rj)
//      (4) vi = vi + (Fi/mi)*(0.5dt)
//      (5) goto (1)
//
//    potential: Lennard-Jones V(r) = 4*epsilon*((sigma/r)^12-(sigma/r)^6)
//           force F(r) = -dV(r)/dr = 24.0*epsilon*r6*(2.0*r6-1.0)/r, r6=(sigma/r)^6
//
//    for fast calculation
//      ignore F(r) r>rCutoff
//      force F(r) <- force table + linear interpolation (see setForceTable() and cutoff(r))
//      register near particles reg[][] (see registerNearParticles()), 'near' means r<rCutoff+20*2000*dt
//        reg[][] use 20 times, assuming particle max speed < 2000m/s
//      force calculation: sum up force(r) (r<rCutoff)
//
// --------------------
*/

const mixLJMD3D = (function(){ // ====================  mixLJMD3D Module  ====================

	const g_AMU = 1.66053904e-27;				// (kg) atomic mass unit
	const g_kB = 1.380649e-23;					// (J/K) Boltzmann's constant
	const g_nMax = 2000;						// array max

	let g_molecKind1 = 2;						// 2:Ar (0:He,1:Ne,2:Ar,3:Kr,4:Xe,5:Hg)
	let g_molecKind2 = 4;						// 4:Xe (0:He,1:Ne,2:Ar,3:Kr,4:Xe,5:Hg)
	let g_nParticles =300;						// number of particles
	let g_sysTime = 0.0;						// (s) system time
	let g_timeStep = 5.0*1.0e-15;				// (s) time step
	let g_xMax = 6.0E-9;						// (m) x-Box size
	let g_yMax = 6.0E-9;						// (m) y-Box size
	let g_zMax = 6.0E-9;						// (m) z-Box size
	let g_kineticEnergy = 0.0;					// (J) total kinetic energy
	let g_potentialEnergy = 0.0;				// (J) total potential energy
	let g_rCutoff = 1.0e-9;						// (m) force cutoff length
	let g_hh = 1.0e-12;							// (m) forceTable r-division

	const g_xx = dim1( g_nMax );				// (m) x-component of i-th particle position
	const g_yy = dim1( g_nMax );				// (m) y-component of i-th particle position
	const g_zz = dim1( g_nMax );				// (m) z-component of i-th particle position
	const g_vx = dim1( g_nMax );				// (m/s) x-component of i-th particle velocity
	const g_vy = dim1( g_nMax );				// (m/s) y-component of i-th particle velocity
	const g_vz = dim1( g_nMax );				// (m/s) z-component of i-th particle velocity
	const g_ffx = dim1( g_nMax );				// (N) x-component of total force applied i-th particle
	const g_ffy = dim1( g_nMax );				// (N) y-component of total force applied i-th particle
	const g_ffz = dim1( g_nMax );				// (N) z-component of total force applied i-th particle
	const g_mas = dim1( g_nMax );				// (kg) mass of i-th particle
	const g_kind = dimInt1( g_nMax );			// kind of i-th particle
	const g_potentialTable = dim3( 6, 6, 1002);	// potentialTable[kind_i][kind_j][i_radius]
	const g_forceTable = dim3( 6, 6, 1002 );	// forceTable[kind_i][kind_j][i_radius]
	const g_reg = dimInt2( g_nMax, 420 );		// reg[i][0]:total number of particles near i-th particle
												// reg[i][j]:particle number near i-th particle, (j>0)

	function dim1( n ) {
		return new Float64Array( n );
	}

	function dimInt1( n ) {
		return new Int32Array( n );
	}

	function dimInt2( ni, nj ) {
		let a = [];
		for (let i=0; i<ni; i++) {
			a[i] = new Int32Array( nj );
		}
		return a;
	}

	function dim3( ni, nj, nk ) {
		let a = [];
		for (let i=0; i<ni; i++) {
			a[i] = [];
			for (let j=0; j<nj; j++) {
				a[i][j] = new Float64Array( nk );
			}
		}
		return a;
	}


	// --------------------  Lennard-Jones potential data  --------------------

	const g_LJMolec = [
	//    mass(kg)        E(J)         sigma(m)   string  color
		[   4.003*g_AMU,  10.2*g_kB, 2.576e-10, "He",  "#ee4444" ], // 0 He
		[  20.183*g_AMU,  36.2*g_kB, 2.976e-10, "Ne",  "#eeaa00" ], // 1 Ne
		[  39.948*g_AMU, 124.0*g_kB, 3.418e-10, "Ar",  "#aadd00" ], // 2 Ar
		[  83.500*g_AMU, 190.0*g_kB, 3.610e-10, "Kr",  "#44dd44" ], // 3 Kr
		[ 131.300*g_AMU, 229.0*g_kB, 4.055e-10, "Xe",  "#0088ee" ], // 4 Xe
		[ 200.590*g_AMU, 851.0*g_kB, 2.898e-10, "Hg",  "#8800ee" ], // 5 Hg
		[   2.016*g_AMU,  33.3*g_kB, 2.968e-10, "H2",  "#ff8800" ], // 6 H2
		[  28.013*g_AMU,  91.5*g_kB, 3.681e-10, "N2",  "#ff0000" ], // 7 N2
		[  31.999*g_AMU, 113.0*g_kB, 3.433e-10, "O2",  "#ff0088" ], // 8 O2
		[  18.015*g_AMU, 809.1*g_kB, 2.641e-10, "H2O", "#ff00ff" ], // 9 H2O
		[  16.043*g_AMU, 137.0*g_kB, 3.822e-10, "CH2", "#bb00ff" ], //10 CH4
		[  44.010*g_AMU, 190.0*g_kB, 3.996e-10, "CO2", "#6600ff" ], //11 CO2
		[  28.011*g_AMU, 110.0*g_kB, 3.590e-10, "CO",  "#0000ff" ]  //12 CO
	];

	const g_rCollision = [];
	const g_rBond = [];
	const g_massOf = [];
	const g_strOf = [];
	const g_colorOf = [];

	(function() {
		const n = g_LJMolec.length;

		for (let i=0; i<n; i++) {
			g_rCollision[i] = 0.5*g_LJMolec[i][2];
			g_rBond[i] = 1.12246*0.5*g_LJMolec[i][2];
			g_massOf[i] = g_LJMolec[i][0];
			g_strOf[i] = g_LJMolec[i][3];
			g_colorOf[i] = g_LJMolec[i][4];
		}
	}());

	function colorStrOf(kind) {
		return "<span style='color:"+g_colorOf[kind]+"'>"+g_strOf[kind]+"</span>";
	}

	//--- set forceTable[kind1][kind2][ir] and potentialTable[kind1][kind2][ir]

	function setForceTable() {
		for (let i=0; i<6; i++) {
			for (let j=0; j<6; j++) {
				const ep = Math.sqrt(g_LJMolec[i][1]*g_LJMolec[j][1]);
				const sg = 0.5*(g_LJMolec[i][2]+g_LJMolec[j][2]);
				for (let ir=1; ir<=1001; ir++) {
					const r = ir*g_hh;
					const ri = (sg/r);
					const r6 = ri*ri*ri*ri*ri*ri;
					g_potentialTable[i][j][ir] = cutoff(r)*4.0*ep*r6*(r6-1.0);
					// V(r) = 4*eps*((sgm/r)^12-(sgm/r)^6)
					// forceTable[i][j][ir] = cutoff(r)*(24.0*ep*r6*(2.0*r6-1.0)/r);
				}
				g_potentialTable[i][j][0] = g_potentialTable[i][j][1] + g_potentialTable[i][j][2];
				for (let ir=1; ir<=1000; ir++) {
					g_forceTable[i][j][ir] = -(g_potentialTable[i][j][ir+1] - g_potentialTable[i][j][ir-1])/(2.0*g_hh);
				}
				g_forceTable[i][j][1001] = -(0.0 - g_potentialTable[i][j][1000])/(2.0*g_hh);
				g_forceTable[i][j][0] = g_forceTable[i][j][1];
			}
		}
	}

	function cutoff(r) {
		let ret = 0.0;
		if (r>0.0 && r<0.8*g_rCutoff) {
			ret = 1.0;
		} else if (r>=0.8*g_rCutoff && r<g_rCutoff ) {
			ret = 0.5+0.5*Math.cos(Math.PI*(r-0.8*g_rCutoff)/(0.2*g_rCutoff));
		}
		return ret;
	}


	// --------------------  set initial condition  --------------------

	function setInitialCondition( theme, nn, BoxSizeInNM, contTemp ) {
		let knd1, knd2, nn1, nn2;

		if (theme==0) { // Ne (knd: 1:Ne,2:Ar,3:Kr,4:Xe,5:Hg)
			knd1 = 1; nn1=nn; knd2 = 1; nn2 = 0;
		} else if (theme==1) { // Ar
			knd1 = 2; nn1=nn; knd2 = 2; nn2 = 0;
		} else if (theme==2) { // Kr
			knd1 = 3; nn1=nn; knd2 = 3; nn2 = 0;
		} else if (theme==3) { // Xe
			knd1 = 4; nn1=nn; knd2 = 4; nn2 = 0;
		} else if (theme==4) { // Hg
			knd1 = 5; nn1=nn; knd2 = 5; nn2 = 0;
		} else if (theme==5) { // Ar-Xe
			knd1 = 2; nn1 = nn/2; knd2 = 4; nn2 = nn/2;
		} else if (theme==6) { // Ar-Hg
			knd1 = 2; nn1 = nn/2; knd2 = 5; nn2 = nn/2;
		} else if (theme==7) { // Ne-Kr
			knd1 = 1; nn1 = nn/2; knd2 = 3; nn2 = nn/2;
		} else if (theme==8) { // Ne-Ar
			knd1 = 1; nn1 = nn/2; knd2 = 2; nn2 = nn/2;
		}

		g_sysTime = 0.0;
		g_molecKind1 = knd1
		g_molecKind2 = knd2
		setForceTable();
		g_xMax = BoxSizeInNM*1.0e-9;
		g_yMax = BoxSizeInNM*1.0e-9;
		g_zMax = BoxSizeInNM*1.0e-9;
		g_nParticles = setGas(knd1,nn1,knd2,nn2);
		ajustVelocity(contTemp);
	}

	function setGas(knd1,nn1,knd2,nn2) {
		let nParticles = nn1+nn2;
		for (let j=0; j<nn1+nn2; j++) {
			let loopCount = 0;
			g_kind[j] = (j<nn1) ? knd1 : knd2;
			let i = 0;
			do {
				const sgm = 2.0*g_rCollision[g_kind[j]];
				g_xx[j] = (g_xMax-1.2*sgm)*Math.random() + 0.6*sgm;
				g_yy[j] = (g_yMax-1.2*sgm)*Math.random() + 0.6*sgm;
				g_zz[j] = (g_zMax-1.2*sgm)*Math.random() + 0.6*sgm;
				for (i=0; i<j; i++) {
					const d = Math.sqrt((g_xx[i]-g_xx[j])*(g_xx[i]-g_xx[j])+(g_yy[i]-g_yy[j])*(g_yy[i]-g_yy[j])
						+(g_zz[i]-g_zz[j])*(g_zz[i]-g_zz[j]));
					if (d < 1.2*(g_rCollision[g_kind[i]]+g_rCollision[g_kind[j]])) break;
				}
				loopCount = loopCount + 1;
				if (loopCount>1000) break;
			} while (i<j);
			if (loopCount>1000) {
				nParticles = j - 1;
				break;
			}
		}

		for (let i=0; i<nParticles; i++) {
			g_vx[i] = 200.0*normalRandom3();
			g_vy[i] = 200.0*normalRandom3();
			g_vz[i] = 200.0*normalRandom3();
			g_ffx[i] = 0.0;
			g_ffy[i] = 0.0;
			g_ffz[i] = 0.0;
			g_mas[i] = g_massOf[g_kind[i]];
		}
		return nParticles;
	}

	// normal distributed random number: -3.0 <= normalRandom3() < 3.0
	function normalRandom3() {
		return (Math.random()+Math.random()+Math.random()+Math.random()+Math.random()+Math.random()-3.0);
	}


	// --------------------  time evolution  --------------------

	function timeEvolution( tempMode, contTemp ) {

		if (tempMode==1) ajustVelocity(contTemp);
		registerNearParticles();
		for (let i=1; i<=20; i++) {
			g_sysTime += g_timeStep;
			moveParticles(g_timeStep);
		}
	}

	function moveParticles(dt) {
		const nn=g_nParticles;

		for (let i=0; i<nn; i++) {
			const a = 0.5*dt/g_mas[i];
			g_vx[i] += a*g_ffx[i];
			g_vy[i] += a*g_ffy[i];
			g_vz[i] += a*g_ffz[i];
			g_xx[i] += g_vx[i]*dt;
			g_yy[i] += g_vy[i]*dt;
			g_zz[i] += g_vz[i]*dt;
		}
		calcForce();
		for (let i=0; i<nn; i++) {
			const a = 0.5*dt/g_mas[i];
			g_vx[i] += a*g_ffx[i];
			g_vy[i] += a*g_ffy[i];
			g_vz[i] += a*g_ffz[i];
		}
		g_kineticEnergy = 0.0;
		for (let i=0; i<nn; i++) {
			g_kineticEnergy += 0.5*g_mas[i]*(g_vx[i]*g_vx[i]+g_vy[i]*g_vy[i]+g_vz[i]*g_vz[i]);
		}
	}

	function calcForce() {
		const nn=g_nParticles, s05 = 0.5*3.418e-10; // Ar sigma=3.418e-10
		g_potentialEnergy = 0.0;
		for (let i=0; i<nn; i++) {
			g_ffx[i]=0;g_ffy[i]=0;g_ffz[i]=0;
		}
		for (let i=0; i<nn-1; i++) {
			for (let k=1; k<g_reg[i][0]; k++) {
				const j = g_reg[i][k];
				const xij=g_xx[i]-g_xx[j];
				const yij=g_yy[i]-g_yy[j];
				const zij=g_zz[i]-g_zz[j];
				const rij = Math.sqrt(xij*xij+yij*yij+zij*zij);
				if (rij<g_rCutoff) {
					const f = force(rij,g_kind[i],g_kind[j]);
					const fxij = f*xij/rij;
					const fyij = f*yij/rij;
					const fzij = f*zij/rij;
					g_ffx[i] += fxij;
					g_ffy[i] += fyij;
					g_ffz[i] += fzij;
					g_ffx[j] -= fxij;
					g_ffy[j] -= fyij;
					g_ffz[j] -= fzij;
				}
			}
		}
		for (let i=0; i<nn; i++) {
			g_ffx[i] += boundaryForce(g_xx[i]+s05)+boundaryForce(g_xx[i]-g_xMax-s05);
			g_ffy[i] += boundaryForce(g_yy[i]+s05)+boundaryForce(g_yy[i]-g_yMax-s05);
			g_ffz[i] += boundaryForce(g_zz[i]+s05)+boundaryForce(g_zz[i]-g_zMax-s05);
		}
	}

	function force(r,ki,kj) { // forceTable - linear interporation
		const hh=g_hh;

		const ir = Math.floor(r/hh);
		const a = r - ir*hh;
		g_potentialEnergy += ((hh-a)*g_potentialTable[ki][kj][ir] + a*g_potentialTable[ki][kj][ir+1])/hh;
		return ((hh-a)*g_forceTable[ki][kj][ir] + a*g_forceTable[ki][kj][ir+1])/hh;
	}

	function boundaryForce(r) {
		const ri = (3.418e-10/r);
		const r6 = ri*ri*ri*ri*ri*ri;
		g_potentialEnergy += 4.0*0.5*1.711e-21*r6*(r6-1.0);
		return (24.0*0.5*1.711e-21*r6*(2.0*r6-1.0)/r);
	}

	//--- registeration

	function registerNearParticles() {
		const nn=g_nParticles;

		const rCut = g_rCutoff+20*2000*g_timeStep;
		const rcut2 = rCut*rCut;
		for (let i=0; i<nn-1; i++) {
			let k = 1;
			for (let j=i+1; j<nn; j++) {
				const r2 = (g_xx[i]-g_xx[j])*(g_xx[i]-g_xx[j])+(g_yy[i]-g_yy[j])*(g_yy[i]-g_yy[j])
						+(g_zz[i]-g_zz[j])*(g_zz[i]-g_zz[j]);
				if (r2<rcut2) {
					g_reg[i][k] = j;
					k = k + 1;
				}
			}
			g_reg[i][0] = k;
		}
	}

	function maxNearParticles() {
		const nn=g_nParticles;
		let mx=0;

		for (let i=0; i<nn-1; i++) {
			if (mx<g_reg[i][0]) mx = g_reg[i][0];
		}
		return (mx-1);
	}


	// --------------------  utility  --------------------

	function systemTemperature() {
		const nn=g_nParticles;
		let ek=0.0;  //ek:kinetic energy (J)
		for (let i=0; i<nn; i++) {
			ek = ek + 0.5*g_mas[i]*(g_vx[i]*g_vx[i]+g_vy[i]*g_vy[i]+g_vz[i]*g_vz[i]);
		}
		return 2.0*ek/(3.0*nn*g_kB); //2D: E/N=kT, 3D: E/N=(3/2)kT
	}

	function ajustVelocity(temp) {
		const nn=g_nParticles;
		const a = Math.sqrt(temp/systemTemperature());
		for (let i=0; i<nn; i++) {
			g_vx[i] = a*g_vx[i];
			g_vy[i] = a*g_vy[i];
			g_vz[i] = a*g_vz[i];
		}
	}


	// --------------------  public  --------------------

	return {
		init:			setInitialCondition,	// setInitialCondition( theme, nn, BoxSizeInNM, contTemp )
		evolve:			timeEvolution,			// timeEvolution( tempMode, contTemp )

		setTemp:		ajustVelocity,			// ajustVelocity( temp )

		getSysParam:	function() {
							return [ g_molecKind1, g_molecKind2, g_nParticles, g_timeStep, g_xMax, g_yMax, g_zMax ];
						},
		getNow:			function() { return [ g_sysTime, systemTemperature(), g_kineticEnergy, g_potentialEnergy ]; },
		getKindStr:		function(kind) { return g_strOf[kind]; },

		getMolecData:	function(i) { const kind = g_kind[i]; return [ kind, g_rCollision[kind] ]; },
		getPosition:	function(i) { return [ g_xx[i], g_yy[i], g_zz[i] ]; },
		getVelocity:	function(i) { return [ g_vx[i], g_vy[i], g_vz[i] ]; },
		getForce:		function(i) { return [ g_ffx[i], g_ffy[i], g_ffz[i] ]; },
	};

})(); // ====================  mixLJMD3D end  ====================


const js012 = (function(){ // ====================  js Module  ====================

	const theModule = mixLJMD3D;
	const xCanvasSize = 480;	// in pixel
	const yCanvasSize = 480;	// in pixel
	let canvas;
	let ctx;

	let v_theme = 8;
	let v_nn = 200;
	let v_BoxSizeInNM = 6.0;
	let v_tempMode = 0;
	let v_contTemp = 300.0;

	let p_kind1, p_kind2, p_nParticles, p_timeStep, p_xMax, p_yMax, p_zMax; // = theModule.getSysParam();
	let sysTime, sysTemp, kineticEnergy, potentialEnergy;

	let nCalc = 2;
	let dispMode = 0;
	let resetFlag = true;
	let pauseFlag = false;
	let stepFlag = false;
  let breakFlag = false;


	function main() {
		theModule.init( v_theme, v_nn, v_BoxSizeInNM, v_contTemp ); // ( nn, BoxSizeInNM, contTemp )
		setCanvas( 'canvas_box', xCanvasSize, yCanvasSize );
		initDom();

		animate();

		function setCanvas( canvasID, width, height ) {
			canvas = document.getElementById( canvasID );
			canvas.width  = width;
			canvas.height = height;
			ctx = canvas.getContext('2d');
			ctx.font = "16px 'sans-serif'";
			ctx.textBaseline = "bottom";
			ctx.textAlign = "left";
			ctx.lineWidth = 1;
			g3d.setMouseOnCanvas( canvas ); // 3D graphics
		}
	}


	function animate() {
    if ( breakFlag ) return;

		if ( resetFlag ) {
			resetFlag = false;
			theModule.init( v_theme, v_nn, v_BoxSizeInNM, v_contTemp ); // ( nn, BoxSizeInNM, contTemp )
			[ p_kind1, p_kind2, p_nParticles, p_timeStep, p_xMax, p_yMax, p_zMax ] = theModule.getSysParam();
			// g3d.init( nParticles, xMax, yMax, zMax, xCanvasSize, yCanvasSize, xBoxSize, yShift )
			g3d.init( p_nParticles, p_xMax, p_yMax, p_zMax, xCanvasSize, yCanvasSize, 300, 20 );
		}

		if ( !pauseFlag ) {
			for(let i=0; i<nCalc; i++ ) {
				theModule.evolve( v_tempMode, v_contTemp );
			}
		} else if ( pauseFlag && stepFlag ) {
			stepFlag = false;
			theModule.evolve( v_tempMode, v_contTemp );
		}

		draw( ctx, dispMode );

		requestAnimationFrame(animate);
	}


	// --------------------  draw  --------------------

	const molecColor = [ /*0:He*/ [ 0xff, 0x50, 0x50 ], /*1:Ne*/ [ 0xff, 0xb4, 0x00 ], /*2:Ar*/ [ 0xb4, 0xff, 0x00 ],
						 /*3:Kr*/ [ 0x50, 0xff, 0x50 ], /*4:Xe*/ [ 0x50, 0xb4, 0xff ], /*5:Hg*/ [ 0x80, 0x00, 0xff ] ];

	function draw( ctx, dispMode ) {
		[ sysTime, sysTemp, kineticEnergy, potentialEnergy ] = theModule.getNow();

		// clear
		ctx.clearRect(0, 0, xCanvasSize, yCanvasSize);

		// draw
		draw3D( ctx, p_nParticles );

		// caption
		const yTextPos = yCanvasSize-45; // in pixel
		ctx.fillStyle = "#888888";
		ctx.fillText(`time =${(sysTime*1.0e12).toFixed(1)} (ps)`, 40, yTextPos);
		ctx.fillText(`Temp =${sysTemp.toFixed(1)} (K)`, 260, yTextPos);

		ctx.fillStyle = ballColor( molecColor[ p_kind1 ], 1.0 );
		ctx.fillText(`${theModule.getKindStr(p_kind1)}`, 40, yTextPos+20);
		if ( p_kind1!=p_kind2 ) {
			ctx.fillStyle = ballColor( molecColor[ p_kind2 ], 1.0 );
			ctx.fillText(`${theModule.getKindStr(p_kind2)}`, 70, yTextPos+20);
		}
		ctx.fillStyle = "#888888";
		ctx.fillText(`,    N = ${p_nParticles}`, 100, yTextPos+20);
		ctx.fillText(`control Temp = ${v_contTemp} (K)`, 260, yTextPos+20);
		ctx.fillText(`Box =${(p_xMax*1.0e9).toFixed(1)}x${(p_yMax*1.0e9).toFixed(1)}x${(p_zMax*1.0e9).toFixed(1)} (nm)`,
						40, yTextPos+40);
		ctx.fillText(`Energy = ${(kineticEnergy+potentialEnergy).toExponential(4)} (J)`, 260, yTextPos+40);
	}


	function draw3D( ctx, nn ) {
		let sc, xp, yp; [ sc, xp, yp ] = g3d.scxpypFunc(); // g3d sc, xp, yp

		// draw 3D
		g3d.set3DRotatedObjects(0.0); //(rotateRateOfAyInDegree) eg. 0.0 or 0.5
		g3d.plotFarEdge( ctx, sc,xp,yp, "#444400" ); // dark yellow

		for (let i=0; i<nn; i++) {
			const j = g3d.srtzix[i];
			let kind, rCollision; [ kind, rCollision ] = theModule.getMolecData(j);
			const mColor = molecColor[ kind ];
			const zr = (g3d.ppz[j]/g_zMax+1.27)/3.0; // -0.73 < g3d.ppz[j]/g_zMax < 1.73
			const x = g3d.ppx[j]*sc+xp, y = g3d.ppy[j]*sc+yp, r = rCollision*sc;
			g3d.drawDisc( ctx, x, y, r, ballColor( mColor, zr ) );
			g3d.strokeCircle( ctx, x, y, r, ballColor( mColor, 0.7*zr ) );
		}

		g3d.plotNearEdge( ctx, sc,xp,yp, "#999900" ); // yellow
		ctx.font = "12px 'sans-serif'";
		ctx.fillStyle = "#888888";
		ctx.fillText(`Ax=${(g3d.Ax*180/Math.PI).toFixed(1)}, Ay=${(g3d.Ay*180/Math.PI).toFixed(1)}`, 10, 15);
		ctx.font = "16px 'sans-serif'";
	}

	function ballColor( ballColorV3, zr ) {
		const r = Math.floor(ballColorV3[0]*zr);
		const g = Math.floor(ballColorV3[1]*zr);
		const b = Math.floor(ballColorV3[2]*zr);
		return `rgb(${r},${g},${b})`;
	}


	// --------------------  3D graphics (particles) module  --------------------
	//
	// ver 0.0.1  2018.12.23  last updated on 2023.04.10
	// ver 0.0.2  2023.05.02  last updated on 2023.06.02

	let g_nParticles, g_xMax, g_yMax, g_zMax, g_xCanvasSize, g_yCanvasSize, g_xBoxSize, g_yShift; // gloval in g3d

	const g3d = {};				// namespace of graphic 3D module
	g3d.mouseDownFlag = 0;		// 1:on mouse down, 0:else
	g3d.x_mouse = 0;			// x-position of mouse
	g3d.y_mouse = 0;			// y-position of mouse
	g3d.x0_mouse = 0;			// drag-started x-position of mouse
	g3d.y0_mouse = 0;			// drag-started y-position of mouse
	g3d.zoom = 1.0;

	g3d.cx0 = 0.0;				// x-rotate center (3D graphics)
	g3d.cy0 = 0.0;				// y-rotate center (3D graphics)
	g3d.cz0 = 0.0;				// z-rotate center (3D graphics)
	g3d.Ax = -Math.PI/12.0;		// x-rotate angle around x-axis
	g3d.Ay = -Math.PI/12.0;		// y-rotate angle around y-axis
	g3d.ddAy = 0.0;				// Ay change rate for auto-rotate: eg. dday=0.5*Math.PI/180
	g3d.cosAx = 0.0;			// cosAx=Math.cos(Ax)
	g3d.sinAx = 0.0;			// sinAx=Math.sin(Ax)
	g3d.cosAy = 0.0;			// cosAy=Math.cos(Ay)
	g3d.sinAy = 0.0;			// sinAy=Math.sin(Ay)

	g3d.ppx = [];				// (m) rotated x-component of i-th particle position
	g3d.ppy = [];				// (m) rotated y-component of i-th particle position
	g3d.ppz = [];				// (m) rotated z-component of i-th particle position
	g3d.srtzix = [];			// z(depth)-sorted index: srtzix[1], srtzix[2],...,srtzix[g_nParticles]

	g3d.xApex = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0];		// x-apex of box
	g3d.yApex = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0];		// y-apex of box
	g3d.zApex = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0];		// z-apex of box
	g3d.pxApex = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0];		// rotated x-apex of box
	g3d.pyApex = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0];		// rotated y-apex of box
	g3d.pzApex = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0];		// rotated z-apex of box
	g3d.boxApex = [[0,0,0], [1,0,0], [0,1,0], [1,1,0], [0,0,1], [1,0,1], [0,1,1], [1,1,1] ];
	g3d.boxEdge = [[0,1,9], [0,2,9], [0,4,9], [1,3,9], [1,5,9], [2,3,9],
				   [2,6,9], [3,7,9], [4,5,9], [4,6,9], [5,7,9], [6,7,9] ];

	//--- set mouse on canvas

	// g3d.setMouseOnCanvas( canvas );
	g3d.setMouseOnCanvas = function( canvas ) {
		canvas.addEventListener('mousemove', g3d.mouse_move);
		canvas.addEventListener('mousedown', g3d.mouse_down);
		canvas.addEventListener('mouseup', g3d.mouse_up);
		//canvas.addEventListener("mousewheel", g3d.mouseWheel);
	};
	g3d.setMouseOnCanvas.defined = false;

	g3d.mouse_move = function(e) {
		const pi = Math.PI;

		if (g3d.mouseDownFlag==1) {
			g3d.x_mouse = e.clientX;
			g3d.y_mouse = e.clientY;
			g3d.Ay = g3d.Ay + 0.5*(g3d.x_mouse-g3d.x0_mouse)*pi/180;
			if (g3d.Ay<-pi) g3d.Ay += 2*pi;
			if (g3d.Ay>pi) g3d.Ay += -2*pi;
			//g3d.Ax = g3d.Ax + 0.5*(g3d.y_mouse-g3d.y0_mouse)*pi/180;
			g3d.Ax = g3d.Ax - 0.5*(g3d.y_mouse-g3d.y0_mouse)*pi/180;
			if (g3d.Ax<-0.5*pi) g3d.Ax = -0.5*pi;
			if (g3d.Ax>0.5*pi) g3d.Ax = 0.5*pi;
			g3d.x0_mouse = g3d.x_mouse;
			g3d.y0_mouse = g3d.y_mouse;
		}
	};

	g3d.mouse_down = function(e) {
		if (g3d.mouseDownFlag==0) {
			g3d.x0_mouse = e.clientX;
			g3d.y0_mouse = e.clientY;
			g3d.x_mouse = g3d.x0_mouse;
			g3d.y_mouse = g3d.y0_mouse;
			g3d.mouseDownFlag = 1;
		}
	};

	g3d.mouse_up = function(e) {
		if (g3d.mouseDownFlag==1) {
			g3d.mouseDownFlag = 0;
		}
	};

	g3d.mouseWheel = function(e) {
		g3d.deltaY = e.deltaY;
		if ( g3d.deltaY > 0 ) g3d.zoom *= 0.95;
		else if ( g3d.deltaY < 0 ) g3d.zoom *= 1.05;
		if ( g3d.zoom<0.5 ) g3d.zoom = 0.5;
		if ( g3d.zoom>2.0 ) g3d.zoom = 2.0;
	};

	//--- 3D graphics aid (particles)

	// g3d.init( nParticles, xMax, yMax, zMax, xCanvasSize, yCanvasSize, xBoxSize, yShift )
	g3d.init = function( nParticles, xMax, yMax, zMax, xCanvasSize, yCanvasSize, xBoxSize, yShift ) {
		g_nParticles = nParticles;
		g_xMax = xMax;
		g_yMax = yMax;
		g_zMax = zMax;
		g_xCanvasSize = xCanvasSize; g_yCanvasSize = yCanvasSize;
		g_xBoxSize = (xBoxSize==undefined) ? 300 : xBoxSize;
		g_yShift = (yShift==undefined) ? 20 : yShift;
		g3d.cx0 = 0.5*g_xMax;		// x-rotate center (3D graphics)
		g3d.cy0 = 0.5*g_yMax;		// y-rotate center (3D graphics)
		g3d.cz0 = 0.5*g_zMax;		// z-rotate center (3D graphics)
	}

	// g3d.setRotateAngle( AxInDegree, AyInDegree );
	g3d.setRotateAngle = function( AxInDegree, AyInDegree ) {
		g3d.Ax = AxInDegree*Math.PI/180.0;
		g3d.Ay = AyInDegree*Math.PI/180.0;
	};

	// g3d.scxpypFunc();
	g3d.scxpypFunc = function() {
		const xBoxSize = g_xBoxSize;
		const xCenter = xCanvasSize/2, yCenter = yCanvasSize/2-g_yShift, yBoxSize = (xBoxSize/g_xMax)*g_yMax;
		const xp = xCenter - (xBoxSize/2)*g3d.zoom, yp = yCenter - (yBoxSize/2)*g3d.zoom; // g3d param
		const sc = (xBoxSize/g_xMax)*g3d.zoom;
		return [ sc, xp, yp ];
	}

	g3d.set3DRotatedObjects = function(rotateRateOfAyInDegree) {
		g3d.ddAy = rotateRateOfAyInDegree*Math.PI/180.0;
		g3d.Ay= g3d.Ay + g3d.ddAy; // auto-rotate : if (ddAy==0.0), stop
		if (g3d.Ay>Math.PI) g3d.Ay = g3d.Ay - 2.0*Math.PI;
		if (g3d.Ay<-Math.PI) g3d.Ay = g3d.Ay + 2.0*Math.PI;
		g3d.setBox();           // set box apex
		g3d.setRotateXY(g3d.Ax,g3d.Ay); // set rotate angle and rotate center(=box center)
		g3d.rotateXY();         // g_xx(i),g_yy(i),g_zz(i) rotate--> ppx(i),ppy(i),ppz(i)
		g3d.sortz();            // sort ppz(i) : ppz(srtzix(1)),ppz(srtzix(2)),...,ppz(srtzix(g_nParticles))
		g3d.markFarEdge();      // boxEdge[iEdge][2]=1:far side edge or 0:near side edge
	};

	g3d.setBox = function() {
		for (let i=0; i<8; i++) {
			g3d.xApex[i] = g3d.boxApex[i][0]*g_xMax;
			g3d.yApex[i] = g3d.boxApex[i][1]*g_yMax;
			g3d.zApex[i] = g3d.boxApex[i][2]*g_zMax;
		}
	};

	g3d.setRotateXY = function(angleX,angleY) {
		g3d.cosAx = Math.cos(angleX);
		g3d.sinAx = Math.sin(angleX);
		g3d.cosAy = Math.cos(angleY);
		g3d.sinAy = Math.sin(angleY);
		g3d.cx0 = 0.5*g_xMax;
		g3d.cy0 = 0.5*g_yMax;
		g3d.cz0 = 0.5*g_zMax;
	};

	g3d.rotateXY = function() { //particles and box apex
		const nn = g_nParticles;
		const cosAx=g3d.cosAx,sinAx=g3d.sinAx,cosAy=g3d.cosAy,sinAy=g3d.sinAy,cx0=g3d.cx0,cy0=g3d.cy0,cz0=g3d.cz0;

		for (let i=0; i<nn; i++) {
			let xi, yi, zi;
			[ xi, yi, zi ] = theModule.getPosition(i);
			const y = g_yMax-yi-cy0;
			g3d.ppx[i] = cosAy*(xi-cx0)+sinAy*(sinAx*y+cosAx*(zi-cz0)) + cx0;
			g3d.ppy[i] = cosAx*y-sinAx*(zi-cz0) + cy0;
			g3d.ppz[i] =-sinAy*(xi-cx0)+cosAy*(sinAx*y+cosAx*(zi-cz0)) + cz0;
		}
		for (let i=0; i<8; i++) {
			g3d.pxApex[i] = cosAy*(g3d.xApex[i]-cx0)+sinAy*(sinAx*(g3d.yApex[i]-cy0)+cosAx*(g3d.zApex[i]-cz0))+cx0;
			g3d.pyApex[i] = cosAx*(g3d.yApex[i]-cy0)-sinAx*(g3d.zApex[i]-cz0) + cy0;
			g3d.pzApex[i] =-sinAy*(g3d.xApex[i]-cx0)+cosAy*(sinAx*(g3d.yApex[i]-cy0)+cosAx*(g3d.zApex[i]-cz0))+cz0;
		}
	};

	g3d.rotateXYPos = function(xi,yi,zi) {
		const cosAx=g3d.cosAx,sinAx=g3d.sinAx,cosAy=g3d.cosAy,sinAy=g3d.sinAy,cx0=g3d.cx0,cy0=g3d.cy0,cz0=g3d.cz0;
		const y = g_yMax-yi-cy0;
		const rotx = cosAy*(xi-cx0)+sinAy*(sinAx*y+cosAx*(zi-cz0)) + cx0;
		const roty = cosAx*y-sinAx*(zi-cz0) + cy0;
		//const rotz =-sinAy*(xi-cx0)+cosAy*(sinAx*y+cosAx*(zi-cz0)) + cz0;
		return [ rotx, roty ];
	};

	g3d.sortz = function() {
		const nn=g_nParticles;
		for (let i=0; i<nn; i++) {
			g3d.srtzix[i] = i;
		}
		g3d.qSort(0,nn-1);
	};

	g3d.qSort = function(le,ri) {
		let i,j, pv,w;
		if (ri>le) {
			i = le-1;
			j = ri;
			pv = g3d.ppz[g3d.srtzix[ri]];
			while (1) {
				do {
					i=i+1;
				} while (pv>g3d.ppz[g3d.srtzix[i]]);
				do {
					j=j-1;
				} while (j>i && g3d.ppz[g3d.srtzix[j]]>pv);
				if (j<=i) break;
				w=g3d.srtzix[i]; g3d.srtzix[i]=g3d.srtzix[j]; g3d.srtzix[j]=w;
			}
			w=g3d.srtzix[i]; g3d.srtzix[i]=g3d.srtzix[ri]; g3d.srtzix[ri]=w;
			g3d.qSort(le,i-1);
			g3d.qSort(i+1,ri);
		}
	};

	g3d.markFarEdge = function() {
		//seek far apex --> iMin
		let zMin = g3d.pzApex[0];
		let iMin = 0;
		for (let i=1; i<8; i++) {
			if (zMin>g3d.pzApex[i]) {
				zMin = g3d.pzApex[i];
				iMin = i;
			}
		}
		//mark far edge
		for (let iEdge=0; iEdge<12; iEdge++) {
			g3d.boxEdge[iEdge][2] = 0;
			if (g3d.boxEdge[iEdge][0]==iMin || g3d.boxEdge[iEdge][1]==iMin) g3d.boxEdge[iEdge][2] = 1;
		}
	};

	g3d.plotNearEdge = function(ctx, sc,xp,yp,color) {
		for (let iEdge=0; iEdge<12; iEdge++) {
			if (g3d.boxEdge[iEdge][2]==0) { //far edge mark == 1
				g3d.plotEdge(ctx, iEdge,sc,xp,yp,color);
			}
		}
	};

	g3d.plotFarEdge = function(ctx, sc,xp,yp,color) {
		for (let iEdge=0; iEdge<12; iEdge++) {
			if (g3d.boxEdge[iEdge][2]==1) { //far edge mark == 1
				g3d.plotEdge(ctx, iEdge,sc,xp,yp,color);
			}
		}
	};

	g3d.plotEdge = function(ctx, iEdge,sc,xp,yp,color) {
		let iApex = g3d.boxEdge[iEdge][0];
		const x1 = g3d.pxApex[iApex]*sc+xp;
		const y1 = g3d.pyApex[iApex]*sc+yp;
		iApex = g3d.boxEdge[iEdge][1];
		const x2 = g3d.pxApex[iApex]*sc+xp;
		const y2 = g3d.pyApex[iApex]*sc+yp;
		g3d.drawLine(ctx, x1, y1, x2, y2, color);
	};

	g3d.drawLine = function(ctx, x1, y1, x2, y2, color) {
		ctx.strokeStyle = color;
		ctx.beginPath();
		ctx.moveTo(x1, y1);
		ctx.lineTo(x2, y2);
		ctx.stroke();
	};

	g3d.drawDisc = function( ctx, x, y, r, color ) {
		ctx.fillStyle = color;
		ctx.beginPath();
		ctx.arc(x, y, r, 0, 2*Math.PI, false);
		ctx.fill();
	};

	g3d.strokeCircle = function( ctx, x, y, r, color ) {
		ctx.strokeStyle = color;
		ctx.beginPath();
		ctx.arc(x, y, r, 0, 2*Math.PI, false);
		ctx.stroke();
	}

	// --------------------  3D graphics (particles) module end  --------------------


	// --------------------  dom control

	function initDom() {
		document.getElementById("step_button").style.visibility = "hidden";
	}

	function reset() { resetFlag = true; }

	function pause() {
		let btn = document.getElementById("pause_button");

		pauseFlag = ( pauseFlag==false );
		if ( pauseFlag==false ) btn.innerHTML = "pause"; else btn.innerHTML = "go";

		if ( pauseFlag==true ) {
			document.getElementById("step_button").style.visibility = "visible";
		} else {
			document.getElementById("step_button").style.visibility = "hidden";
		}
	}

	function step() { stepFlag = true; }

	function setTheme() {
		v_theme = 0 + document.getElementById("slct_theme").selectedIndex;
		resetFlag = true;
	}

	function setnParticles() {
		const n = 1 + document.getElementById("slct_nn").selectedIndex;
		v_nn = n*100;
		resetFlag = true;
	}

	function setTempMode() {
		v_tempMode = 0 + document.getElementById("slct_tempMode").selectedIndex;
	}

	function setContTemp() {  // range mixing
		v_contTemp = Number(document.getElementById("range_temp").value);
		document.getElementById("text_temp").innerHTML = " " + v_contTemp.toFixed(0);
	}

	function setDispMode() {
		dispMode = 0 + document.getElementById("slct_dispMode").selectedIndex;
	}

	function setNcalc() {
		nCalc = 1 + document.getElementById("slct_nCalc").selectedIndex;
	}

	function viewHome() {
		g3d.setRotateAngle(-10,-15); // in degree
		g3d.zoom = 1.0;
	}


  // function controlled by python

  function breakLoop() {
    breakFlag = true;
  }

  function pysetTheme( theme ) {
    v_theme = theme
    document.getElementById("slct_theme").selectedIndex = theme;
    resetFlag = true;
  }

	function pysetTempMode( mode ) {
		v_tempMode = mode;
		document.getElementById("slct_tempMode").selectedIndex = mode;
	}

  function pysetTemperature(temp) {
    theModule.setTemp(temp);
  }

  function pysetDispMode( mode ) {
    dispMode = mode;
    document.getElementById("slct_dispMode").selectedIndex = mode;
  }

  function pygetData( pyMsg ) {
    document.getElementById("text_from_python").innerHTML = pyMsg;
    return [ sysTime, sysTemp, kineticEnergy, potentialEnergy ];
  }


	// --------------------  public  --------------------

	return {
		main:			main,			// main()

		// dom control
		reset:			reset,			// reset()
		pause:			pause,			// pause()
		step:			step,			// step()

		setTheme:		setTheme,		// setTheme()
		setnParticles:	setnParticles,	// setnParticles()
		setTempMode:	setTempMode,	// setTempMode()
		setContTemp:	setContTemp,	// setContTemp()
		setDispMode:	setDispMode,	// setDispMode()
		setNcalc:		setNcalc,		// setNcalc()
		viewHome:		viewHome,		// viewHome()

    breakLoop: breakLoop, // breakLoop();
    pysetTheme: pysetTheme, // pysetTheme( theme )
    pysetTempMode: pysetTempMode, // pysetTempMode( mode )
    pysetTemperature: pysetTemperature, // pysetTemperature( temp )
    pysetDispMode: pysetDispMode, // pysetDispMode( mode )
    pygetData: pygetData, // pygetData( pyMsg ) : return [ sysTime, temperature, kineticEnergy, potentialEnergy ]
  };


})(); // ====================  js012 module end  ====================


const js = js012;
js.main();


// %%%%%%%%%%%%%%%%%%%%  end of javaScript  %%%%%%%%%%%%%%%%%%%%

</script>
<style type="text/css">
body { text-align:left; color:#000000; background-color:#fff8dd; }
</style>
</head>

<body>
<p>[js012] mixed gas (Lennard-Jones potential) MD3D</p>
<canvas ID="canvas_box" style="background-color: #000000;" WIDTH="480" HEIGHT="480"></canvas><br>

<label>molecules:</label>
<select id="slct_theme" onChange="js.setTheme()">
<option>Ne</option><option>Ar</option><option>Kr</option><option>Xe</option><option>Hg</option>
<option>Ar-Xe</option><option>Ar-Hg</option><option>Ne-Kr</option><option selected>Ne-Ar</option>
</select>
</select>
    <span style="margin-right: 50px;"></span>
<label>N=</label>
<select id="slct_nn" onChange="js.setnParticles()">
<option>100</option><option selected>200</option><option>300</option>
<option>400</option><option>500</option>
</select>
    <span style="margin-right: 50px;"></span>
<button id="pause_button" onClick="js.pause()">pause</button>
    <span style="margin-right: 10px;"></span>
<button id="step_button" onClick="js.step()">step</button>
<br>

<label>temp mode:</label>
<select id="slct_tempMode" onChange="js.setTempMode()">
<option selected>adiabatic</option><option>Temp.control</option>
</select>
<br>

<label>cont. Temp</label>
<input type="range" id="range_temp" min="10" max="600" value="300" step="2"
style="width:300px" oninput="js.setContTemp()">
    <span style="margin-right: 20px;"></span>
<label id="text_temp"> 300</label>
<br>

<label>speed(nCalc/frame):</label>
<select id="slct_nCalc" onChange="js.setNcalc()">
<option>1</option><option selected>2</option><option>3</option><option>4</option>
</select>
    <span style="margin-right: 100px;"></span>
<button onClick="js.viewHome()">return to initial view</button>
<br>

<p id="text_caption" ></p>
<hr width="480" align="left" color="#a0a0a0">
<button onClick="js.breakLoop()">animation break to END</button>
    <span style="margin-right: 50px;"></span> python msg:
<span id="text_from_python" ></span>
<br>

</body>
</html>


  ''')
  display(htm)
# end def


In [ ]:
# exec html-js code
exec_html_js()
print("--- push [animation break to END] button to end ---")

In [ ]:
# exec html-js code, and python control

import time
import numpy as np
import matplotlib.pyplot as plt


# exec html-js code
exec_html_js()
print("-- start --")
time.sleep(3)

# python control
for i in range(10):
  # get data and display
  [ sysTime, sysTemp, kineticEnergy, potentialEnergy ] = eval_js( 'js.pygetData({})'.format(i) )
  energy = kineticEnergy + potentialEnergy
  print(
    "i = {}, ".format(i),
    "\t time = {:.2f} (ps), ".format(sysTime*1e12),
    "temp = {:.2f} (K), ".format(sysTemp),
    "energy = {:.6g} (J) ".format(energy) )
  time.sleep(3)
#

time.sleep(1)
# animation break to END
eval_js( 'js.breakLoop()' )
print("-- end --")

In [ ]:
# exec html-js code, and python control / change theme

import time
import numpy as np
import matplotlib.pyplot as plt

def setTheme( theme ):
  eval_js('js.pysetTheme({})'.format(theme))

themeList = [ '0:Ne', '1:Ar', '2:Kr', '3:Xe', '4:Hg', '5:Ar-Xe', '6:Ar-Hg', '7:Ne-Kr', '8:Ne-Ar'  ]

# exec html-js code
exec_html_js()
print("-- start --")
time.sleep(3)

# change theme
for theme in [ 1, 4, 5, 8 ]:
  setTheme( theme )
  print("-- theme:", themeList[theme], "--")

  for i in range(4):
    # get data and display
    [ sysTime, sysTemp, kineticEnergy, potentialEnergy ] = eval_js( 'js.pygetData({})'.format(i) )
    energy = kineticEnergy + potentialEnergy
    print(
      "-- -- i = {}, ".format(i),
      "time = {:.2f} (ps), ".format(sysTime*1e12),
      "temp = {:.2f} (K), ".format(sysTemp),
      "energy = {:.6g} (J) ".format(energy) )
    time.sleep(2)
#

time.sleep(1)
# animation break to END
eval_js( 'js.breakLoop()' )
print("-- end --")

In [ ]:
# exec html-js code, and python control / system cool down and plot

import time
import numpy as np
import matplotlib.pyplot as plt


# exec html-js code
exec_html_js()
print("-- start --")
time.sleep(3)

# python control
theme = 1 # Ar
setTheme( theme )
print("-- theme:", themeList[theme], "--")

timeList = []
tempList = []
energyList = []
kkList = []
uuList = []

for i in range(20):
  # set temperature
  temp = 300 - 15*i
  eval_js( 'js.pysetTemperature({})'.format(temp) )
  time.sleep(3)

  # get data and display
  [ sysTime, sysTemp, kineticEnergy, potentialEnergy ] = eval_js( 'js.pygetData({})'.format(i) )
  energy = kineticEnergy + potentialEnergy

  timeList.append(sysTime*1e12)
  tempList.append(sysTemp)
  energyList.append(energy*1e20)
  kkList.append(kineticEnergy*1e20)
  uuList.append(potentialEnergy*1e20)

  if i%2==0:
    print(
      "-- -- i = {}, ".format(i),
      "time = {:.2f} (ps), ".format(sysTime*1e12),
      "temp = {:.2f} (K), ".format(sysTemp),
      "energy = {:.6g} (J) ".format(energy) )
  time.sleep(3)
#
time.sleep(3)


# animation break to END
eval_js( 'js.breakLoop()' )
print("-- end --")

# plot
plt.plot( timeList, tempList, 'or-' )
plt.xlabel('time (ps)')
plt.ylabel('temp (K)')
plt.title("temperture change")
plt.show()

plt.plot( timeList, energyList, 'or-' )
plt.plot( timeList, kkList, 'xg-' )
plt.plot( timeList, uuList, '+b-')
plt.xlabel('time (ps)')
plt.ylabel('energy ( x 10^-20 J )')
plt.title("energy change (energy:red, K:green, U:blue)")
plt.show()